In [1]:
BLOCK_SIZE = 1024

NEALE_SAMPLES_INPUT = 'gs://ukb_data/phenomexcan/samples/samples_neale_eids.csv'
SAMPLES_OUTPUT = 'gs://ukb_data/phenomexcan/samples/samples_neale_eids.ht'

N_SAMPLE = 50000
N_SAMPLE_SEED = 0
SAMPLE_PREFIX = f'{int(N_SAMPLE / 1000)}k'
SAMPLES_SAMPLED_OUTPUT = f'gs://ukb_data/phenomexcan/samples/{SAMPLE_PREFIX}/samples_{SAMPLE_PREFIX}_neale_eids.ht'
display(SAMPLES_SAMPLED_OUTPUT)

'gs://ukb_data/phenomexcan/samples/50k/samples_50k_neale_eids.ht'

In [2]:
import hail as hl
#hl.init(default_reference='GRCh37', min_block_size=1024)
#hl.init(min_block_size=BLOCK_SIZE, tmp_dir='gs://ukb_data/tmp/')
hl.init(tmp_dir='gs://ukb_data/tmp/', min_block_size=BLOCK_SIZE)

Running on Apache Spark version 2.4.3
SparkUI available at http://ukb-m.c.ukb-im.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.21-f16fd64e0d77
LOGGING: writing to /home/hail/hail-20190909-1526-0.2.21-f16fd64e0d77.log


# Create Neale's samples table

In [3]:
samples = (
    hl.import_table(NEALE_SAMPLES_INPUT)
    .key_by('eid')
    .write(SAMPLES_OUTPUT, overwrite=True)
)

2019-09-09 15:26:17 Hail: INFO: Reading table with no type imputation
  Loading column 'eid' as type 'str' (type not specified)

2019-09-09 15:26:20 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-09-09 15:26:35 Hail: INFO: wrote table with 361202 rows in 1 partition to gs://ukb_data/phenomexcan/samples/samples_neale_eids.ht


# Sample samples to compute LD

In [4]:
# read samples
samples = hl.read_table(SAMPLES_OUTPUT)
n_neale_samples = samples.count()
print(n_neale_samples)

361202


In [5]:
perc = N_SAMPLE / (n_neale_samples * 1.0)
print(perc)
samples_downsampled = samples.sample(perc, seed=N_SAMPLE_SEED)

0.13842669752659176


In [6]:
samples_downsampled.write(SAMPLES_SAMPLED_OUTPUT, overwrite=True)

2019-09-09 15:26:57 Hail: INFO: wrote table with 50581 rows in 1 partition to gs://ukb_data/phenomexcan/samples/50k/samples_50k_neale_eids.ht
